# Create material files

In [ ]:
import openmc
import numpy as np
import matplotlib.pyplot as plt

################################################################################
#Materials
################################################################################
#Plutonium (PuBe)
PuBe = openmc.Material(name = 'PuBe')
PuBe.add_nuclide('Pu239', 1.0,  'ao')     
PuBe.add_element('Be',    13.0, 'ao') 
PuBe.set_density('g/cm3', 15.0) 

################################################################################
#Aluminum Cladding
aluminum_cladding = openmc.Material(name = 'Aluminum Cladding')
aluminum_cladding.add_element('Al', 1.0)
aluminum_cladding.set_density('g/cm3', 2.7)

################################################################################
#Fuel (UO2)
fuel = openmc.Material(name='UO2 Fuel')
fuel.add_nuclide('U238', 0.99284)
fuel.add_nuclide('U235', 0.00711)
fuel.add_nuclide('U234', 0.00005)
fuel.add_s_alpha_beta('c_O_in_UO2')
fuel.add_element('O', 2)
fuel.set_density('g/cm3', 10.5)

################################################################################
#Graphite Moderator
graphite = openmc.Material(name='Graphite')
graphite.add_element('C', 1.0)
graphite.set_density('g/cm3', 1.7)
graphite.add_s_alpha_beta('c_Graphite')

################################################################################
#Air
air = openmc.Material(name='Air')
air.add_element('N',  0.78084)
air.add_element('O',  0.209476)
air.add_element('Ar', 0.00934)
air.set_density('g/cm3', 0.001204)

################################################################################
#Concrete
concrete = openmc.Material(name='Concrete')
concrete.add_element('H',  0.149857)
concrete.add_element('C',  0.074206)
concrete.add_element('O',  0.526836)
concrete.add_element('Mg', 0.017713)
concrete.add_element('Al', 0.023794)
concrete.add_element('Si', 0.091975)
concrete.add_element('S',  0.001649)
concrete.add_element('K',  0.000773)
concrete.add_element('Ca', 0.109681)
concrete.add_element('Fe', 0.003516)
concrete.set_density('g/cm3', 2.35)

################################################################################
#Export Materials
materials = openmc.Materials([fuel, PuBe, aluminum_cladding, graphite, air, concrete])
materials.export_to_xml()

# Create Main Dimensions of Reactor

In [ ]:
################################################################################
#Brick dimensions in the x-y-z planes
brick_side_length_x  = 15
brick_side_length_y  = 15
brick_side_length_z  = 50.0

################################################################################
# Radii
r_air_hole   = 0.46
r_inner_clad = 0.61875
r_fuel       = 1.76125
r_outer_clad = 1.92

# Lengths
total_length = 135.0
air_gap_back = 15.0

#Dimensions in total:
x_direction = 165.24
y_direction = 150
z_direction = 150

# Create Reactor Boundary and Fuel Universe

In [ ]:
# Reactor Boundary Conditions
x_min = openmc.XPlane(x0=0,      boundary_type='vacuum')
x_max = openmc.XPlane(x0=165.24, boundary_type='vacuum')
y_min = openmc.YPlane(y0=-30,    boundary_type='vacuum')
y_max = openmc.YPlane(y0=150.0,  boundary_type='vacuum')
z_min = openmc.ZPlane(z0=0,      boundary_type='vacuum')
z_max = openmc.ZPlane(z0=150.0,  boundary_type='vacuum')

# Combine the boundary planes to form a cuboid region
reactor_boundary = (+x_min & -x_max & +y_min & -y_max & +z_min & -z_max)

############################################################################
# Make Fuel Universe

# Define the fuel region with a single cylinder per hole
z_plane_start = openmc.ZPlane(z0=0)
z_plane_air   = openmc.ZPlane(z0=total_length)
z_plane_end   = openmc.ZPlane(z0=z_direction)

# Define cylinders for air hole, cladding, and fuel
air_hole   = openmc.ZCylinder(r=r_air_hole,   boundary_type='transmission')
inner_clad = openmc.ZCylinder(r=r_inner_clad, boundary_type='transmission')
fuel_rod   = openmc.ZCylinder(r=r_fuel,       boundary_type='transmission')
outer_clad = openmc.ZCylinder(r=r_outer_clad, boundary_type='transmission')

# Define cells for air hole, cladding, and fuel
air_hole_cell = openmc.Cell(name='Air Hole', fill=air)
air_hole_cell.region = -air_hole & +z_plane_start & -z_plane_air

inner_clad_cell = openmc.Cell(name='Inner Cladding', fill=aluminum_cladding)
inner_clad_cell.region = +air_hole & -inner_clad & +z_plane_start & -z_plane_air

fuel_cell = openmc.Cell(name='Fuel Rod', fill=fuel)
fuel_cell.region = +inner_clad & -fuel_rod & +z_plane_start & -z_plane_air

outer_clad_cell = openmc.Cell(name='Outer Cladding', fill=aluminum_cladding)
outer_clad_cell.region = +fuel_rod & -outer_clad & +z_plane_start & -z_plane_air

#Don't forget about the 15 cm air gap on the back
air_cylinder = openmc.ZCylinder(r=r_outer_clad, boundary_type='transmission')
air_cell = openmc.Cell(name='Air Gap', fill=air)
air_cell.region = -air_cylinder & +z_plane_air & -z_plane_end 

pitch = 15.0
#Graphite Bricks
graphite_bricks = openmc.model.rectangular_prism(width=pitch, height=pitch)
graphite_bricks_cell = openmc.Cell(name='Graphite_bricks')
graphite_bricks_cell.fill = graphite
graphite_bricks_cell.region = graphite_bricks & +air_cylinder & +outer_clad & -z_plane_end & +z_min

fuel_rod_universe = openmc.Universe(cells=[air_hole_cell, inner_clad_cell, fuel_cell, outer_clad_cell, air_cell, graphite_bricks_cell])

### Visualize Fuel

In [ ]:
fuel_rod_universe.plot(width=(20,20), colors = {fuel_cell:'green', air_hole_cell:'blue', graphite_bricks_cell:'black', inner_clad_cell:'grey', outer_clad_cell:'red', air_cell:'yellow'}, basis=('xy'))

# Create Fuel Lattice Structure

In [ ]:
quarter_pitch = pitch*10
fuel_universe_lattice = openmc.RectLattice(name='Fuel Assembly')
fuel_universe_lattice.pitch = (pitch, pitch)
fuel_universe_lattice.lower_left = (0, 0)  # Centered in a 150x150 box


# Make a 10x10 lattice for the fuel
fuel_universe_lattice.universes = [
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe],
    [fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe, fuel_rod_universe]
]

fuel_assembly_region = openmc.model.rectangular_prism(width=quarter_pitch, height=quarter_pitch, origin=(75, 75))
fuel_assembly_cell = openmc.Cell(name='quarter assembly cell', fill=fuel_universe_lattice, region=fuel_assembly_region)

quarter_assembly_universe = openmc.Universe(cells=[fuel_assembly_cell])

### Visualize the Lattice Structure

In [ ]:
quarter_assembly_universe.plot(width=(200,200), colors = {fuel_cell:'green', air_hole_cell:'blue', graphite_bricks_cell:'black', inner_clad_cell:'grey', outer_clad_cell:'red', air_cell:'yellow'}, origin=(75.0, 75.0, 75.0), basis=('xy'))

# Create Graphite Floor and Plutonium Source

In [ ]:
plutonium_region = openmc.Sphere(r=2.0, x0=75.0, y0=-15.0, z0=75.0)
plutonium_cell = openmc.Cell(fill=PuBe, region=-plutonium_region)

graphite_floor_region = openmc.model.RectangularParallelepiped(
    xmin=0, xmax=150, ymin=-30, ymax=0, zmin=0, zmax=150
)
graphite_floor_cell = openmc.Cell(fill=graphite, region=-graphite_floor_region & +plutonium_region)

graphite_floor_universe = openmc.Universe(cells=[plutonium_cell, graphite_floor_cell])

### Visualize the Graphite Floor and Plutonium Fuel

In [ ]:
graphite_floor_universe.plot(width=(100,100), colors = {graphite_floor_cell:'black', air_cell:'yellow'}, basis=('xy'))

# Create Concrete Wall

In [ ]:
concrete_wall_region = openmc.model.RectangularParallelepiped(
    xmin=150, xmax=165.24, ymin=-30, ymax=150, zmin=0, zmax=150
)
concrete_wall_cell = openmc.Cell(fill=concrete, region=-concrete_wall_region)

concrete_wall_universe = openmc.Universe(cells=[concrete_wall_cell])

### Visualize the Concrete Wall

In [ ]:
concrete_wall_universe.plot(width=(100,100), colors = {concrete_wall_cell:'grey'}, basis=('xy'))

# Visualizing the Whole Geometry and Combining Universes

In [ ]:
# Combine All Universes
universe_cells = [fuel_assembly_cell, plutonium_cell, graphite_floor_cell, concrete_wall_cell]
root_universe = openmc.Universe(cells=[openmc.Cell(fill=openmc.Universe(cells=universe_cells), region=reactor_boundary)])

geometry = openmc.Geometry(root_universe)
geometry.export_to_xml()

material_colors = {
    PuBe: 'yellow',                # Plutonium-Beryllium in yellow
    aluminum_cladding: 'grey',     # Aluminum Cladding in grey
    fuel: 'green',                 # Fuel (UO2) in green
    graphite: 'black',             # Graphite in black
    air: 'blue',                   # Air in blue
    concrete: 'brown'              # Concrete in brown
}

plot_xy = openmc.Plot()
plot_xy.basis = 'xy'
plot_xy.origin = (75.0, 90.0, 75.0)
plot_xy.width = (300., 300.)
plot_xy.pixels = (4000, 4000)
plot_xy.color_by = 'material'
plot_xy.colors = material_colors

plot_xz = openmc.Plot()
plot_xz.basis = 'xz'
plot_xz.origin = (75.0, 90.0, 75.0)
plot_xz.width = (150., 150.)
plot_xz.pixels = (4000, 4000)
plot_xz.color_by = 'material'
plot_xz.colors = material_colors

plot_yz = openmc.Plot()
plot_yz.basis = 'yz'
plot_yz.origin = (75.0, 90.0, 75.0)
plot_yz.width = (150., 150.)
plot_yz.pixels = (4000, 4000)
plot_yz.color_by = 'material'
plot_yz.colors = material_colors

plots = openmc.Plots([plot_xy, plot_xz, plot_yz])
plots.export_to_xml()
openmc.plot_geometry()

# Simulation Settings

In [ ]:
# Make the initial guess start uniformly within the fuel lattice
x_min, x_max = 0.0,   150.0
y_min, y_max = -30.0, 150.0
z_min, z_max = 0.0,   150.0

x_distribution = openmc.stats.Uniform(x_min, x_max)
y_distribution = openmc.stats.Uniform(y_min, y_max)
z_distribution = openmc.stats.Uniform(z_min, z_max)

spatial_distribution = openmc.stats.CartesianIndependent(x_distribution, y_distribution, z_distribution)

source = openmc.IndependentSource()
source.space = spatial_distribution
source.angle = openmc.stats.Isotropic()
source.strength = 1e8

################################################################################
# Setup simulation settings
settings = openmc.Settings()
settings.source = source
settings.batches = 15
settings.inactive = 5
settings.particles = 10000000
settings.export_to_xml()
settings.create_fission_neutrons = True
################################################################################
# Define a mesh that spans the x-y-z planes
mesh = openmc.RegularMesh()
mesh.dimension = [300, 300, 300]
mesh.lower_left = [0.0, -30.0, 0.0]
mesh.upper_right = [150.0, 150.0, 150.0]

# Running the Simulation, Flux, Fission, and Shannon Entropy

In [ ]:
# Define a mesh filter
mesh_filter = openmc.MeshFilter(mesh)

################################################################################
# Define a tally for the flux
flux_tally = openmc.Tally(name='Flux')
flux_tally.filters = [mesh_filter]
flux_tally.scores = ['flux']

################################################################################
# Define a tally for the fission
fission_tally = openmc.Tally(name='Fission')
fission_tally.filters = [mesh_filter]
fission_tally.scores = ['fission']

################################################################################
# Add both tallies to a single tallies collection
tallies = openmc.Tallies()
tallies.append(flux_tally)
tallies.append(fission_tally)

# Export tallies to XML
tallies.export_to_xml()

################################################################################
# Create shannon entropy mesh
entropy_mesh = openmc.RegularMesh()
entropy_mesh.dimension = [8, 8, 8]  # Adjust dimensions as needed
entropy_mesh.lower_left = [x_min, y_min, z_min]
entropy_mesh.upper_right = [x_max, y_max, z_max]
settings.entropy_mesh = entropy_mesh

################################################################################
# Assemble and Run
model = openmc.model.Model(geometry, materials, settings, tallies=tallies)
model.run(threads=18, geometry_debug=False)


# Post Processing, Visualizing $k_{eff}$ Convergence and Shannon Entropy

In [ ]:
import numpy as np
import openmc
import matplotlib.pyplot as plt

# ----------------------------------------------------------------------------
# 1) Load statepoint and basic parameters
# ----------------------------------------------------------------------------
sp = openmc.StatePoint('statepoint.15.h5')

n_inactive = sp.n_inactive
n_batches  = sp.n_batches
n_active   = n_batches - n_inactive

# ----------------------------------------------------------------------------
# 2) Generation-based data: raw k_generation + Shannon entropy
# ----------------------------------------------------------------------------
k_eff_raw = np.array(sp.k_generation)  # one k-eff per generation
H         = np.array(sp.entropy)       # one Shannon entropy per generation

n_generations = len(k_eff_raw)
gens = np.arange(1, n_generations + 1)

# ----------------------------------------------------------------------------
# 3) Batch-based data: average each batch from k_generation
# ----------------------------------------------------------------------------
# Determine how many generations per batch (assumes even division)
gens_per_batch = n_generations // n_batches

# Average each row => one k value per batch
k_per_batch = np.mean(k_eff_raw.reshape(n_batches, gens_per_batch), axis=1)

# Compute running average over active batches
batch_mean   = np.full(n_batches, np.nan)
batch_stderr = np.full(n_batches, np.nan)

for i in range(n_batches):
    if i < n_inactive:
        # Skip inactive
        continue
    else:
        active_slice = k_per_batch[n_inactive : i+1]
        batch_mean[i] = np.mean(active_slice)
        if len(active_slice) > 1:
            std_sample = np.std(active_slice, ddof=1)
            batch_stderr[i] = std_sample / np.sqrt(len(active_slice))
        else:
            batch_stderr[i] = 0.0

# Plot each batch point at the generation index where the batch ends
batch_x = (np.arange(n_batches) + 1) * gens_per_batch

# ----------------------------------------------------------------------------
# 4) Create the figure and plot
# ----------------------------------------------------------------------------
fig, ax1 = plt.subplots()

# (a) Plot raw generation-based k (no error bars)
line1, = ax1.plot(
    gens, k_eff_raw, 'o-', label='Gen-based k', alpha=0.7
)

# (b) Plot batch-based running average k (with error bars)
line2 = ax1.errorbar(
    batch_x, batch_mean, yerr=batch_stderr,
    fmt='s-', capsize=3, label='Batch-based k', alpha=0.7
)

# Left y-axis for k
ax1.set_xlabel('Generation')
ax1.set_ylabel('k-effective', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True)

# (c) Plot Shannon entropy on a second y-axis
ax2 = ax1.twinx()
line3, = ax2.plot(
    gens, H, 'r-', label='Shannon entropy'
)
ax2.set_ylabel('Shannon Entropy', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# ----------------------------------------------------------------------------
# 5) Combine legends and place them on the middle right
# ----------------------------------------------------------------------------
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()

# Merge them into a single legend
ax2.legend(lines1 + lines2, labels1 + labels2, loc='center right')

plt.title('Gen-based k, Batch-based k, and Shannon Entropy')
plt.tight_layout()
plt.show()


# Post Processing, Visualizing Flux and Fission

In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Function to get data based on tally and score
def get_tally_data(tally_name, score='fission'):
    tally = sp.get_tally(name=tally_name)
    return tally.get_values(scores=[score])

# Retrieve the fission data (change tally name and score if needed)
data_type = 'fission'  # Change this to 'flux', 'fission', or any other score available
plot_title = 'Fission Events'  # Adjust title for visualization
tally_name = 'Fission'  # Adjust the tally name if necessary
data = get_tally_data(tally_name=tally_name, score=data_type)

# Define the mesh size and reshape the data
total_data_points = 300**3
mesh_dimension_size = np.round(np.cbrt(total_data_points)).astype(int)
data_reshaped = data.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Determine the data range for consistent color scaling
data_min = data_reshaped.min()
data_max = data_reshaped.max()

# Function to display a single slice
def show_slice(z_slice_index):
    plt.figure(figsize=(8, 6))
    data_2d = data_reshaped[z_slice_index, :, :]

    # Generate the heatmap for the x-y slice with fixed color scale
    plt.imshow(data_2d, origin='lower', cmap='nipy_spectral', vmin=data_min, vmax=data_max)
    plt.colorbar(label='Fission Events')
    plt.title(f'Fission Events in the x-y plane at z index {z_slice_index}')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.show()

# Create an interactive slider
interact(show_slice, z_slice_index=IntSlider(min=0, max=mesh_dimension_size-1, step=1, value=0));

#Up the contrast of the color bars to make it nicer to look at. 


In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Function to get data based on tally and score
def get_tally_data(tally_name, score='flux'):
    tally = sp.get_tally(name=tally_name)
    return tally.get_values(scores=[score])

# Retrieve the fission data (change tally name and score if needed)
data_type = 'flux'  # Change this to 'flux', 'fission', or any other score available
plot_title = 'Neutron Flux'  # Adjust title for visualization
tally_name = 'Flux'  # Adjust the tally name if necessary
data = get_tally_data(tally_name=tally_name, score=data_type)

# Define the mesh size and reshape the data
total_data_points = 300**3
mesh_dimension_size = np.round(np.cbrt(total_data_points)).astype(int)
data_reshaped = data.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Determine the data range for consistent color scaling
data_min = data_reshaped.min()
data_max = data_reshaped.max()

# Function to display a single slice
def show_slice(z_slice_index):
    plt.figure(figsize=(8, 6))
    data_2d = data_reshaped[z_slice_index, :, :]

    # Generate the heatmap for the x-y slice with fixed color scale
    plt.imshow(data_2d, origin='lower', cmap='nipy_spectral', vmin=data_min, vmax=data_max)
    plt.colorbar(label='Neutron Flux')
    plt.title(f'{plot_title} in the x-y plane at z index {z_slice_index}')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.show()

# Create an interactive slider
interact(show_slice, z_slice_index=IntSlider(min=0, max=mesh_dimension_size-1, step=1, value=0));


# Create a gif File Showing the Fission or Flux

In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation, PillowWriter

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Function to get data based on tally and score
def get_tally_data(tally_name, score='flux'):
    tally = sp.get_tally(name=tally_name)
    return tally.get_values(scores=[score])

# Retrieve the fission data (change tally name and score if needed)
data_type = 'flux'  # Change this to 'flux', 'fission', or any other score available
plot_title = 'Neutron Flux'  # Adjust title for visualization
tally_name = 'Flux'  # Adjust the tally name if necessary
data = get_tally_data(tally_name=tally_name, score=data_type)

# Define the mesh size and reshape the data
total_data_points = 300**3
mesh_dimension_size = np.round(np.cbrt(total_data_points)).astype(int)
data_reshaped = data.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Determine the data range for consistent color scaling
data_min = data_reshaped.min()
data_max = data_reshaped.max()

# Initialize the figure and axis
fig, ax = plt.subplots()

# Function to update the animation for each frame (z-slice)
def update(z_slice_index, plot_title):
    ax.clear()  # Clear previous frame
    data_2d = data_reshaped[z_slice_index, :, :]
    
    # Generate the heatmap for the x-y slice
    im = ax.imshow(data_2d, origin='lower', cmap='viridis', vmin=data_min, vmax=data_max)
    ax.set_title(f'{plot_title} in the x-y plane at z index {z_slice_index}')
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    return im,

custom_title = "Neutron Flux"

# Create the animation
num_frames = mesh_dimension_size  # One frame per z-slice
ani = FuncAnimation(fig, update, frames=num_frames, fargs=(custom_title,), blit=True, interval=100)

# Save the animation as a video
ani.save('neutron_flux_animation.gif', writer='ffmpeg', fps=10)

# Optionally display the animation as HTML
from IPython.display import HTML
HTML(ani.to_jshtml())

# Close the plot to clean up
plt.close()

# Visualizing Errors

In [ ]:
import openmc
import matplotlib.pyplot as plt
import numpy as np

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Access the flux tally
flux_tally = sp.get_tally(name='Flux')

# Get the mean and standard deviation for the flux tally
flux_mean = flux_tally.get_values(scores=['flux'], value='mean')
flux_std_dev = flux_tally.get_values(scores=['flux'], value='std_dev')

# Total number of data points
total_data_points = 300**3

# Since the data is from a cubic mesh, we take the cube root to find the size of one dimension
mesh_dimension_size = np.round(np.cbrt(total_data_points)).astype(int)

# Reshape the mean and std_dev data to the mesh dimensions
flux_mean_reshaped = flux_mean.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))
flux_std_dev_reshaped = flux_std_dev.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Compute the relative error
relative_error = np.zeros_like(flux_mean_reshaped)
nonzero = flux_mean_reshaped > 0
relative_error[nonzero] = flux_std_dev_reshaped[nonzero] / flux_mean_reshaped[nonzero]

# Plot the distribution of relative errors
plt.hist(relative_error[nonzero], bins=500)
plt.title("Distribution of Relative Errors (Flux)")
plt.xlabel("Relative Error")
plt.ylabel("Frequency")
plt.show()

# Access the fission tally
fission_tally = sp.get_tally(name='Fission')

# Get the mean and standard deviation for the fission tally
fission_mean = fission_tally.get_values(scores=['fission'], value='mean')
fission_std_dev = fission_tally.get_values(scores=['fission'], value='std_dev')

# Reshape the mean and std_dev data to the mesh dimensions
fission_mean_reshaped = fission_mean.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))
fission_std_dev_reshaped = fission_std_dev.reshape((mesh_dimension_size, mesh_dimension_size, mesh_dimension_size))

# Compute the relative error
relative_error_fission = np.zeros_like(fission_mean_reshaped)
nonzero_fission = fission_mean_reshaped > 0
relative_error_fission[nonzero_fission] = fission_std_dev_reshaped[nonzero_fission] / fission_mean_reshaped[nonzero_fission]

# Plot the distribution of relative errors for fission
plt.hist(relative_error_fission[nonzero_fission], bins=500)
plt.title("Distribution of Relative Errors (Fission)")
plt.xlabel("Relative Error")
plt.ylabel("Frequency")
plt.show()


# Plotting Heat Map of Errors

In [ ]:
import openmc
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Load the statepoint file
sp = openmc.StatePoint('statepoint.15.h5')

# Access the tallies
flux_tally = sp.get_tally(name='Flux')
fission_tally = sp.get_tally(name='Fission')

# Extract mean and standard deviation for flux and fission
flux_data_mean = flux_tally.get_values(scores=['flux'], value='mean')
fission_data_mean = fission_tally.get_values(scores=['fission'], value='mean')
flux_data_std_dev = flux_tally.get_values(scores=['flux'], value='std_dev')
fission_data_std_dev = fission_tally.get_values(scores=['fission'], value='std_dev')

# Define the mesh dimensions (assumed cubic with 300 points in each dimension)
data_points = 300

# Reshape the 1D arrays into 3D volumes
flux_data_mean = flux_data_mean.reshape((data_points, data_points, data_points))
fission_data_mean = fission_data_mean.reshape((data_points, data_points, data_points))
flux_data_std_dev = flux_data_std_dev.reshape((data_points, data_points, data_points))
fission_data_std_dev = fission_data_std_dev.reshape((data_points, data_points, data_points))

# Calculate the relative errors (std_dev / mean)
Relative_errors_flux = flux_data_std_dev / flux_data_mean
Relative_errors_fission = fission_data_std_dev / fission_data_mean

# Convert to percentage
flux_error_percent = Relative_errors_flux * 100
fission_error_percent = Relative_errors_fission * 100

vmin = 0
vmax = 100

# Determine a common color scale range for both images
data_min = min(Relative_errors_flux.min(), Relative_errors_fission.min())
data_max = max(Relative_errors_flux.max(), Relative_errors_fission.max())

def show_slice(z_slice_index):
    # Create a figure with two subplots side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Rotate the slices to orient them correctly for display
    flux_slice = np.rot90(flux_error_percent[z_slice_index, :, :], 4)
    fission_slice = np.rot90(fission_error_percent[z_slice_index, :, :], 4)
    
    # Plot flux relative error
    im1 = axes[0].imshow(flux_slice, origin='lower', cmap='nipy_spectral', vmin=vmin, vmax=vmax)
    axes[0].set_title(f'Flux Relative Error (Slice {z_slice_index})')
    axes[0].set_xlabel('X-axis')
    axes[0].set_ylabel('Y-axis')
    plt.colorbar(im1, ax=axes[0])
    
    # Plot fission relative error
    im2 = axes[1].imshow(fission_slice, origin='lower', cmap='nipy_spectral', vmin=vmin, vmax=vmax)
    axes[1].set_title(f'Fission Relative Error (Slice {z_slice_index})')
    axes[1].set_xlabel('X-axis')
    axes[1].set_ylabel('Y-axis')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()

# Create an interactive slider to select the z-slice
interact(show_slice, z_slice_index=IntSlider(min=0, max=data_points-1, step=1, value=data_points//2));
